In [55]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [56]:
states = gpd.read_file("https://raw.githubusercontent.com/GeoDaCenter/covid/master/public/geojson/state_usafacts.geojson")[["GEOID","NAME"]]
states["NAME"] = states["NAME"].str.replace(' ', '-').str.lower()
state_dict = states.to_dict(orient="Records")

In [57]:
# format of the state dict
state_dict[0]

{'GEOID': '23', 'NAME': 'maine'}

In [58]:
counties = gpd.read_file('./counties/cb_2018_us_county_500k.shp')
counties = counties.to_crs("EPSG:5070")

In [52]:
results = []

for state in state_dict:
    print(f"Parsing {state['NAME']}...")
    county_data = counties[counties.STATEFP == state['GEOID']]
    try:
        state_data = gpd.read_file(f"{state['NAME']}_dissolved.gpkg").to_crs("EPSG:5070")
        for i in tqdm(range(0, len(county_data))):
            results.append({
                'GEOID': county_data.iloc[i].GEOID
                'area': gpd.clip(state_data, county_data.iloc[i].geometry).area.sum()
            })
    except:
        print('could not find file')
        
pd.DataFrame(results).to_csv('area_results.csv', index=False)

maine


100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [02:44<00:00, 10.28s/it]
